In [1]:
import spacy
import re
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
from spacy.tokens import Span
from nav_pii_anon.spacy.spacy_model import SpacyModel
from nav_pii_anon.regex_engine import fnr
from nav_pii_anon.regex_engine.tlfnr import RegexTlfNr
from nav_pii_anon.regex_engine.date_time import RegexDateTime

## Using the Entity Ruler object from SpaCy

In [11]:
tx="Jeg er Robindra Prabhu og mitt fødselsnummer er 15044216652 og tlf er +47 99 11 92 34 . Jeg kommer innom 20.06.2020 for å \
snakke om min 100 prosent stilling 99119234 og horse"
type(tx)

str

### Assigning multiple custom entities

In [3]:
dist_checker = fnr.RegexFnr()

dist = dist_checker.validate_pnr('170415443149')

The Levenstein distance from a pnr is 1.0


In [4]:
TRAIN_DATA = [
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, 'TLF')]},
    ),
    ("horses pretend to care about your feelings", {"entities": [(0, 6, 'TLF')]}),
    (
        "they pretend to care about your feelings, those horses",
        {"entities": [(48, 54, 'TLF')]},
    ),
    ("horses?", {"entities": [(0, 6, 'TLF')]}),
]

In [5]:
nlp = SpacyModel()
nlp.train(TRAIN_DATA, output_dir='/Users/petternymark/Documents/nav-pii-anonymizer/examples/test_models')
nlp.add_patterns()

Losses {'ner': 34.97676526068912}
Losses {'ner': 40.53613889678552}
Losses {'ner': 40.27108380443826}
Losses {'ner': 38.038651329188724}
Losses {'ner': 37.08086164496763}
Losses {'ner': 31.904704843836953}
Losses {'ner': 31.581508613191545}
Losses {'ner': 30.96950707898941}
Losses {'ner': 33.36337713323883}
Losses {'ner': 22.171053479600232}
Losses {'ner': 26.818434871303907}
Losses {'ner': 31.376203121872095}
Losses {'ner': 23.506829102228494}
Losses {'ner': 15.308410263684024}
Losses {'ner': 27.92943842290242}
Losses {'ner': 27.205743839598654}
Losses {'ner': 38.980519693621645}
Losses {'ner': 26.057576783804734}
Losses {'ner': 36.1572113659538}
Losses {'ner': 32.79715998383472}
Losses {'ner': 32.84084349425393}
Losses {'ner': 25.713549013192733}
Losses {'ner': 30.259312246316767}
Losses {'ner': 31.240643865405936}
Losses {'ner': 28.717516958326087}
Losses {'ner': 20.804327259779086}
Losses {'ner': 26.086928586501685}
Losses {'ner': 26.307454260498655}
Losses {'ner': 26.1387335574993

In [12]:
from spacy import displacy
doc = nlp.get_doc(tx)
displacy.render(doc, style='ent', jupyter=True)

(15044216652,)
(15044216652, +47 99 11 92 34)
(15044216652, +47 99 11 92 34, 99119234)
(15044216652, +47 99 11 92 34, 20.06.2020, 99119234)
(15044216652, +47 99 11 92 34, 20.06.2020, 100 prosent, 99119234)


In [7]:
doc = nlp.get_doc('99929212')

(99929212,)


In [8]:
doc.ents[0].label_

'TLF'